In [11]:
from bokeh.plotting import figure, curdoc
from bokeh.models import ColumnDataSource, DatetimeTickFormatter
from bokeh.layouts import column
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from functools import partial
from tornado import gen

# Initial sample data
now = datetime.now()
dates = [now + timedelta(seconds=i) for i in range(100)]
prices = np.cumsum(np.random.normal(0, 1, 100)) + 100
volumes = np.abs(np.random.normal(0, 1, 100) * 1000) + 500

# Create DataFrames with initial data
df = pd.DataFrame({
    'date': dates,
    'price': prices,
    'volume': volumes
})

# Create a ColumnDataSource
source = ColumnDataSource(df)

# Price chart with WebGL for hardware acceleration
p1 = figure(height=400, width=800, title="Price Chart (Real-Time)", 
          x_axis_type="datetime", output_backend="webgl")
p1.line('date', 'price', source=source, line_width=2)
p1.xaxis.formatter = DatetimeTickFormatter(
    minutes="%H:%M:%S",
    hourmin="%H:%M:%S",
    hours="%H:%M:%S")

# Volume chart with WebGL
p2 = figure(height=200, width=800, title="Volume", 
          x_axis_type="datetime", output_backend="webgl")
p2.vbar('date', width=timedelta(seconds=0.8), top='volume', source=source, alpha=0.6)
p2.xaxis.formatter = DatetimeTickFormatter(
    minutes="%H:%M:%S",
    hourmin="%H:%M:%S",
    hours="%H:%M:%S")

# Create callback for streaming data
@gen.coroutine
def update():
    # Simulate new data point (in real app, fetch from market API)
    new_time = source.data['date'][-1] + timedelta(seconds=1)
    last_price = source.data['price'][-1]
    new_price = last_price + np.random.normal(0, 1)
    new_volume = abs(np.random.normal(0, 1) * 1000) + 500
    
    # Update data source
    new_data = {
        'date': [new_time],
        'price': [new_price],
        'volume': [new_volume]
    }
    source.stream(new_data, rollover=300)  # Keep only last 300 points

# Add the layout to curdoc
curdoc().add_root(column(p1, p2))
curdoc().add_periodic_callback(update, 100)  # Update every 100ms
curdoc().title = "Real-Time Trading Dashboard"